In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split

In [ ]:
# Load the dataset
train_dir = '/content/drive/MyDrive/cats_vs_dogs_small/train'
val_dir = '/content/drive/MyDrive/cats_vs_dogs_small/validation'
test_dir = '/content/drive/MyDrive/cats_vs_dogs_small/test'
image_size = (224, 224)  # resize images to this size


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Using Small Dataset

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=image_size,
                                                    batch_size=32,
                                                    class_mode='binary')

val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory(val_dir,
                                                target_size=image_size,
                                                batch_size=32,
                                                class_mode='binary')

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=image_size,
                                                  batch_size=32,
                                                  class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Define the CNN model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define early stopping criteria
early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

# Train the model
history = model.fit(train_generator, epochs=20, validation_data=val_generator, callbacks=[early_stop])

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)

print('Test accuracy:', test_acc)


Epoch 1/20
63/63 [==============================] - 422s 7s/step - loss: 0.7657 - accuracy: 0.5130 - val_loss: 0.6900 - val_accuracy: 0.5280
Epoch 2/20
63/63 [==============================] - 14s 223ms/step - loss: 0.6798 - accuracy: 0.5845 - val_loss: 0.6692 - val_accuracy: 0.5940
Epoch 3/20
63/63 [==============================] - 14s 220ms/step - loss: 0.6488 - accuracy: 0.6210 - val_loss: 0.6474 - val_accuracy: 0.6320
Epoch 4/20
63/63 [==============================] - 14s 219ms/step - loss: 0.6035 - accuracy: 0.6815 - val_loss: 0.6114 - val_accuracy: 0.6610
Epoch 5/20
63/63 [==============================] - 14s 218ms/step - loss: 0.5533 - accuracy: 0.7170 - val_loss: 0.6080 - val_accuracy: 0.6830
Epoch 6/20
63/63 [==============================] - 14s 216ms/step - loss: 0.4695 - accuracy: 0.7765 - val_loss: 0.5662 - val_accuracy: 0.7180
Epoch 7/20
63/63 [==============================] - 14s 221ms/step - loss: 0.3863 - accuracy: 0.8260 - val_loss: 0.6344 - val_accuracy: 0.7040
E

In [ ]:
print('Test accuracy:', test_acc)

Test accuracy: 0.6959999799728394


# Pretrained Model

In [ ]:

from keras.applications.vgg16 import VGG16

In [ ]:
conv_base = VGG16(
    weights='imagenet',
    include_top = False,
    input_shape=(image_size[0], image_size[1], 3)
)

In [ ]:
model_1 = Sequential()

model_1.add(conv_base)
model_1.add(Flatten())
model_1.add(Dense(256,activation='relu'))
model_1.add(Dense(1,activation='sigmoid'))

In [ ]:
conv_base.trainable = False

In [ ]:
model_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model_1.fit(train_generator, epochs=10, validation_data=val_generator)



Epoch 1/10
63/63 [==============================] - 20s 293ms/step - loss: 0.7668 - accuracy: 0.7925 - val_loss: 0.2532 - val_accuracy: 0.8910
Epoch 2/10
63/63 [==============================] - 15s 234ms/step - loss: 0.1456 - accuracy: 0.9495 - val_loss: 0.2035 - val_accuracy: 0.9180
Epoch 3/10
63/63 [==============================] - 15s 232ms/step - loss: 0.0833 - accuracy: 0.9820 - val_loss: 0.2323 - val_accuracy: 0.9040
Epoch 4/10
63/63 [==============================] - 15s 233ms/step - loss: 0.0421 - accuracy: 0.9945 - val_loss: 0.2025 - val_accuracy: 0.9150
Epoch 5/10
63/63 [==============================] - 15s 234ms/step - loss: 0.0257 - accuracy: 0.9990 - val_loss: 0.2557 - val_accuracy: 0.8990
Epoch 6/10
63/63 [==============================] - 15s 234ms/step - loss: 0.0156 - accuracy: 1.0000 - val_loss: 0.2073 - val_accuracy: 0.9190
Epoch 7/10
63/63 [==============================] - 15s 234ms/step - loss: 0.0114 - accuracy: 1.0000 - val_loss: 0.2112 - val_accuracy: 0.9190

In [ ]:
# Evaluate the model on the test set
test_loss, test_acc = model_1.evaluate(test_generator)

print('Test accuracy:', test_acc)

32/32 [==============================] - 5s 153ms/step - loss: 0.2194 - accuracy: 0.9190
Test accuracy: 0.9190000295639038


In [ ]:
print('Test accuracy:', test_acc)

Test accuracy: 0.9190000295639038



# Using Kaggle Data Set